In [1]:
# IMPORT PACKAGES
from astropy.io import fits # FITS I/O
import ctypes # C/C++/CUDA shared libraries
import numpy as np # NumPy
from pysiril.siril import Siril # Siril app
from pysiril.wrapper import Wrapper # Siril API
import time # Timing

## CPU Code (Python calling Siril)

In [2]:
%%capture
# LAUNCH SIRIL
app = Siril("/usr/bin/siril")
cmd = Wrapper(app)
app.Open()

INFO   : VERSION siril 1.2.0 : 
INFO   : Siril is compatible with pysiril
INFO   : Initialisation pySiril V0.0.15 : OK
INFO   : First step: Starting 'Siril -p' ....
INFO   : ThreadSiril is started
INFO   : run : /usr/bin/siril -p
INFO   : waiting: 
7s 
INFO   : Second step: Starting pipe reader ....
INFO   : ThreadReader is started
INFO   : Third step: Starting pipe writer ....
INFO   : PipeWriter is started
INFO   : Open()
> requires 1.2.0
: ready
: log: Running command: requires
> [status: success]


True

In [7]:
%%capture
cmd.cd("/home/aanishpradhan/GitHub/cuSiril/assets/NGC-1499/Bias/")
cmd.stack("Bias", type = "rej", rejection_type = "sigma", sigma_low = 3, 
	sigma_high = 3, norm = "no", output_norm = False, rgb_equal = False, 
	out = "Bias_stacked.fit")

## GPU Code (Python calling CUDA)

In [8]:
# SHARED LIBRARIES
clib = ctypes.CDLL("/home/aanishpradhan/GitHub/cuSiril/src/Stacking/stack.so")

## Class/Type Definitions
class Stack(ctypes.Structure):
    _fields_ = [
        ("numberOfSubframes", ctypes.c_uint64),
        ("imageWidth", ctypes.c_uint64),
        ("imageHeight", ctypes.c_uint64),
        ("pixelsPerImage", ctypes.c_uint64),
        ("redSubframes", ctypes.POINTER(ctypes.c_uint16)),
        ("greenSubframes", ctypes.POINTER(ctypes.c_uint16)),
        ("blueSubframes", ctypes.POINTER(ctypes.c_uint16)),
        ("stackedRed", ctypes.POINTER(ctypes.c_float)),
        ("stackedGreen", ctypes.POINTER(ctypes.c_float)),
        ("stackedBlue", ctypes.POINTER(ctypes.c_float))
    ]

In [9]:
# DATA INGESTION
frames = fits.open("/home/aanishpradhan/GitHub/cuSiril/assets/NGC-1499/Bias/Bias.fit")

In [10]:
# DATA TRANSFORMATION

def flattenSubframes(subframes: list) -> list:
	"""
	Flattens a list of subframes.
	
	Args:
		subframes (list): A list of Astropy FITS HDU objects which contain the 
		individual subframes.
	Returns:
		A list of image statistics and NumPy ndarrays. The list contains, in 
		order, the following: number of subframes, image width, image height, 
		flattened and appended red color channels from each subframe, 
		flattened and appended green color channels from each subframe, 
		flattened and appended blue color channels from each subframe
	"""

	# Determine how much memory to allocate
	imageWidth = subframes[0].data.T.shape[0]
	imageHeight = subframes[0].data.T.shape[1]
	pixelsPerImage = imageWidth * imageHeight
	numberOfSubframes = len(subframes)
	dataType = subframes[0].data.dtype
	print("Image width:", subframes[0].data.T.shape[0])
	print("Image height:", subframes[0].data.T.shape[1])
	print("Pixels per image:", pixelsPerImage)
	print("Number of subframes:", numberOfSubframes)
	print("Data type:", dataType)
	
	# Allocate empty arrays
	flattenedRedChannels = np.empty((1, pixelsPerImage * numberOfSubframes), dtype = dataType)
	flattenedGreenChannels = np.empty((1, pixelsPerImage * numberOfSubframes), dtype = dataType)
	flattenedBlueChannels = np.empty((1, pixelsPerImage * numberOfSubframes), dtype = dataType)

	# Flatten subframes by color channel
	for subframe in range(0, len(subframes)):
		startIndex = subframe * pixelsPerImage
		endIndex = (subframe + 1) * pixelsPerImage

		flattenedRedChannels[0, startIndex:endIndex] = subframes[subframe].data.T[:, :, 0].flatten()
		flattenedGreenChannels[0, startIndex:endIndex] = subframes[subframe].data.T[:, :, 1].flatten()
		flattenedBlueChannels[0, startIndex:endIndex] = subframes[subframe].data.T[:, :, 2].flatten()

	return [numberOfSubframes, imageWidth, imageHeight, flattenedRedChannels.flatten(), flattenedGreenChannels.flatten(), flattenedBlueChannels.flatten()]

In [11]:
flattenedBiasFrames = flattenSubframes(frames)

Image width: 4948
Image height: 3280
Pixels per image: 16229440
Number of subframes: 5
Data type: uint16


In [12]:
clib.initializeStackHOST.restype = ctypes.POINTER(Stack)
clib.launchSumStack.restype = ctypes.POINTER(Stack)
clib.launchSigmaClipping.restype = ctypes.POINTER(Stack)

x = clib.initializeStackHOST(ctypes.c_uint64(flattenedBiasFrames[0]),
							 ctypes.c_uint64(flattenedBiasFrames[1]),
							 ctypes.c_uint64(flattenedBiasFrames[2]),
							 ctypes.c_void_p(flattenedBiasFrames[3].ctypes.data),
							 ctypes.c_void_p(flattenedBiasFrames[4].ctypes.data),
							 ctypes.c_void_p(flattenedBiasFrames[5].ctypes.data))

In [13]:
x = clib.launchSigmaClipping(x, ctypes.c_float(3.0), ctypes.c_float(3.0))